In [1]:
import pandas as pd
import pandasql as ps

In [2]:
Categories = pd.read_csv("data/category_data.csv")
Products = pd.read_csv("data/product_data.csv")

## PROBLEM 1

In [3]:
query = """
SELECT
    * FROM Products P
JOIN 
    Categories C on C.category_id = P.category_id
WHERE 
    C.category_id = 8
"""

ps.sqldf(query)

,product_id,product_name,description,price,category_id,category_id,category_name
0,15,Mountain Bike,Conquer the trails with this high-performance ...,1000.0,8,8,Sports & Outdoors
1,16,Tennis Racket,Take your tennis game to the next level with t...,54.0,8,8,Sports & Outdoors


## PROBLEM 2

In [4]:
Users = pd.read_csv("data/user_data.csv")
Orders = pd.read_csv("data/order_data.csv")

In [5]:
query = """
SELECT
    U.user_id, U.username, O.num_orders
FROM 
    Users as U
JOIN 
    (SELECT user_id, count(distinct order_id) as num_orders
    FROM Orders
    GROUP BY user_id) as O
ON 
    U.user_id = O.user_id
"""

ps.sqldf(query)

,user_id,username,num_orders
0,1,johndoe,1
1,2,janesmith,1
2,3,maryjones,1
3,4,robertbrown,1
4,5,sarahwilson,1
5,6,michaellee,1
6,7,lisawilliams,1
7,8,chrisharris,1
8,9,emilythompson,1
9,10,davidmartinez,1


## PROBLEM 3

In [6]:
Reviews = pd.read_csv("data/review_data.csv")

In [8]:
query = """
-- Problem 3: Retrieve the average rating for each product
-- Write an SQL query to retrieve the average rating for each product.
-- The result should include the product ID, product name, and the average rating.
SELECT
    P.product_id, P.product_name, R.average_rating
FROM 
    Products as P
JOIN 
    (SELECT product_id, AVG(rating) as average_rating
    FROM Reviews
    GROUP BY product_id) as R
ON 
    P.product_id = R.product_id
"""

ps.sqldf(query)

,product_id,product_name,average_rating
0,1,Smartphone X,5.0
1,2,Wireless Headphones,4.0
2,3,Laptop Pro,3.0
3,4,Smart TV,5.0
4,5,Running Shoes,2.0
5,6,Designer Dress,4.0
6,7,Coffee Maker,5.0
7,8,Toaster Oven,3.0
8,9,Action Camera,4.0
9,10,Board Game Collection,1.0


## PROBLEM 4

In [9]:
query = """
-- Problem 4: Retrieve the top 5 users with the highest total amount spent on orders
-- Write an SQL query to retrieve the top 5 users with the highest total amount spent on orders.
-- The result should include the user ID, username, and the total amount spent.

SELECT
    U.user_id, U.username, O.total_amount
FROM 
    Users as U
JOIN 
    (SELECT user_id, SUM(total_amount) as total_amount
    FROM Orders
    GROUP BY user_id) as O
ON 
    U.user_id = O.user_id
ORDER BY
    o.total_amount DESC
LIMIT 5
"""


ps.sqldf(query)


,user_id,username,total_amount
0,12,jasonrodriguez,160.0
1,4,robertbrown,155.0
2,8,chrisharris,150.0
3,24,jamesrogers,150.0
4,17,olivialopez,145.0


## PROBLEM 5

In [12]:
query = """
-- Problem 5: Retrieve the products with the highest average rating
-- Write an SQL query to retrieve the products with the highest average rating.
-- The result should include the product ID, product name, and the average rating.
-- Hint: You may need to use subqueries or common table expressions (CTEs) to solve this problem.

WITH R AS
    (SELECT product_id, AVG(rating) as average_rating
    FROM Reviews
    GROUP BY product_id)
SELECT
    P.product_id, P.product_name, R.average_rating
FROM 
    Products as P
JOIN 
    R
ON 
    P.product_id = R.product_id
WHERE
    R.average_rating = (select max(average_rating) from R)
"""

ps.sqldf(query)

,product_id,product_name,average_rating
0,1,Smartphone X,5.0
1,4,Smart TV,5.0
2,7,Coffee Maker,5.0
3,11,Yoga Mat,5.0
4,15,Mountain Bike,5.0


In [14]:
Order_Items = pd.read_csv("data/order_items_data.csv")

In [37]:
query = """
-- Problem 6: Retrieve the users who have made at least one order in each category
-- Write an SQL query to retrieve the users who have made at least one order in each category.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries or joins to solve this problem.

WITH UserOrderCategories AS (
    SELECT U.user_id, C.category_id
    FROM Users U
    JOIN Orders O ON U.user_id = O.user_id
    JOIN Order_Items OI ON O.order_id = OI.order_id
    JOIN Products P ON OI.product_id = P.product_id
    JOIN Categories C ON P.category_id = C.category_id
    GROUP BY U.user_id, C.category_id
)

SELECT 
    UOC.user_id, U.username
FROM
    Users U
JOIN 
    UserOrderCategories UOC ON U.user_id = UOC.user_id
GROUP BY 
    UOC.user_id, U.username
HAVING 
    COUNT(DISTINCT UOC.category_id) = (SELECT COUNT(*) FROM Categories);
"""

ps.sqldf(query)

,user_id,username


In [41]:
query = """
-- Problem 7: Retrieve the products that have not received any reviews
-- Write an SQL query to retrieve the products that have not received any reviews.
-- The result should include the product ID and product name.
-- Hint: You may need to use subqueries or left joins to solve this problem.

SELECT P.product_id, P.product_name
FROM Products P
LEFT JOIN Reviews R ON P.product_id = R.product_id
-- WHERE R.review_id IS NULL OR R.product_id IS NULL;
"""

ps.sqldf(query)

,product_id,product_name
0,1,Smartphone X
1,2,Wireless Headphones
2,3,Laptop Pro
3,4,Smart TV
4,5,Running Shoes
5,6,Designer Dress
6,7,Coffee Maker
7,8,Toaster Oven
8,9,Action Camera
9,10,Board Game Collection


In [73]:
query = """
-- Problem 8: Retrieve the users who have made consecutive orders on consecutive days
-- Write an SQL query to retrieve the users who have made consecutive orders on consecutive days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries or window functions to solve this problem.

WITH UserOrderDates AS (
    SELECT
        user_id,
        order_id,
        order_date,
        LAG(order_date) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_order_date
    FROM Orders
)

SELECT DISTINCT UOD.user_id, U.username
FROM UserOrderDates UOD
JOIN Users U ON UOD.user_id = U.user_id
WHERE UOD.prev_order_date = DATE_ADD(UOD.order_date, INTERVAL -1 DAY);
"""

ps.sqldf(query)

PandaSQLException: (sqlite3.OperationalError) near "DAY": syntax error
[SQL: 
-- Problem 8: Retrieve the users who have made consecutive orders on consecutive days
-- Write an SQL query to retrieve the users who have made consecutive orders on consecutive days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries or window functions to solve this problem.

WITH UserOrderDates AS (
    SELECT
        user_id,
        order_id,
        order_date,
        LAG(order_date) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_order_date
    FROM Orders
)

SELECT DISTINCT UOD.user_id, U.username
FROM UserOrderDates UOD
JOIN Users U ON UOD.user_id = U.user_id
WHERE UOD.prev_order_date = DATE_ADD(UOD.order_date, INTERVAL -1 DAY);
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [67]:
query = """
-- Problem 9: Retrieve the top 3 categories with the highest total sales amount
-- Write an SQL query to retrieve the top 3 categories with the highest total sales amount.
-- The result should include the category ID, category name, and the total sales amount.
-- Hint: You may need to use subqueries, joins, and aggregate functions to solve this problem.

SELECT
    C.category_id,
    C.category_name,
    SUM(OI.quantity * OI.unit_price) AS total_sales_amount
FROM Categories C
JOIN Products P ON C.category_id = P.category_id
JOIN Order_Items OI ON P.product_id = OI.product_id
GROUP BY C.category_id, C.category_name
ORDER BY total_sales_amount DESC
LIMIT 3;

"""

ps.sqldf(query)

,category_id,category_name,total_sales_amount
0,8,Sports & Outdoors,155.0
1,4,Home & Kitchen,145.0
2,1,Electronics,125.0


In [69]:
query = """
-- Problem 10: Retrieve the users who have placed orders for all products in the Toys & Games
-- Write an SQL query to retrieve the users who have placed orders for all products in the Toys & Games
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries, joins, and aggregate functions to solve this problem.

SELECT U.user_id, U.username
FROM Users U
JOIN Orders O ON U.user_id = O.user_id
JOIN Order_Items OI ON O.order_id = OI.order_id
JOIN Products P ON OI.product_id = P.product_id
JOIN Categories C ON P.category_id = C.category_id
WHERE C.category_name = 'Toys & Games'
GROUP BY U.user_id, U.username
HAVING COUNT(DISTINCT OI.product_id) = (
    SELECT COUNT(*)
    FROM Products
    WHERE category_id = C.category_id
);


"""

ps.sqldf(query)

,user_id,username
0,5,sarahwilson


In [72]:
query = """
-- Problem 11: Retrieve the products that have the highest price within each category
-- Write an SQL query to retrieve the products that have the highest price within each category.
-- The result should include the product ID, product name, category ID, and price.
-- Hint: You may need to use subqueries, joins, and window functions to solve this problem.

WITH RankedProducts AS (
    SELECT
        product_id,
        product_name,
        category_id,
        price,
        RANK() OVER (PARTITION BY category_id ORDER BY price DESC) AS rank
    FROM Products
)

SELECT
    product_id,
    product_name,
    category_id,
    price
FROM RankedProducts
WHERE rank = 1;

"""

ps.sqldf(query)

,product_id,product_name,category_id,price
0,1,Smartphone X,1,500.0
1,3,Laptop Pro,2,1200.0
2,6,Designer Dress,3,300.0
3,7,Coffee Maker,4,80.0
4,9,Action Camera,5,200.0
5,12,Skincare Set,6,150.0
6,14,Weighted Blanket,7,100.0
7,15,Mountain Bike,8,1000.0


In [79]:
query = """
-- Problem 12: Retrieve the users who have placed orders on consecutive days for at least 3 days
-- Write an SQL query to retrieve the users who have placed orders on consecutive days for at least 3 days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries, joins, and window functions to solve this problem.

WITH UserOrderDates AS (
    SELECT
        user_id,
        order_date,
        LAG(order_date, 2) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_order_date
    FROM Orders
)

SELECT DISTINCT UOD.user_id, U.username
FROM UserOrderDates UOD
JOIN Users U ON UOD.user_id = U.user_id
WHERE DATEDIFF(UOD.order_date, UOD.prev_order_date) = 2
   AND UOD.prev_order_date IS NOT NULL
GROUP BY UOD.user_id, U.username;

"""

ps.sqldf(query)

PandaSQLException: (sqlite3.OperationalError) no such function: DATEDIFF
[SQL: 
-- Problem 12: Retrieve the users who have placed orders on consecutive days for at least 3 days
-- Write an SQL query to retrieve the users who have placed orders on consecutive days for at least 3 days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries, joins, and window functions to solve this problem.

WITH UserOrderDates AS (
    SELECT
        user_id,
        order_date,
        LAG(order_date, 2) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_order_date
    FROM Orders
)

SELECT DISTINCT UOD.user_id, U.username
FROM UserOrderDates UOD
JOIN Users U ON UOD.user_id = U.user_id
WHERE DATEDIFF(UOD.order_date, UOD.prev_order_date) = 2
   AND UOD.prev_order_date IS NOT NULL
GROUP BY UOD.user_id, U.username;



]
(Background on this error at: https://sqlalche.me/e/20/e3q8)